In [8]:
import os
import sys
import datetime
from random import sample
from pathlib import Path
import girder_client
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np

from Spine/ultrasound_batch_generator import train_preprocess, train_preprocess_with_maps, generate_weight_maps
import evaluation_metrics

from Spine/models import (
    new_unet,
    weighted_categorical_crossentropy,
    weighted_categorical_crossentropy_with_maps,
)
import utils


batch_size=128, 
num_epochs=100.0, 
sagittal_only=False, 
num_frames=1, 
with_maps=False, 
learning_rate=0.002,
lr_decay=False,
dropout=0.0,
use_attention=True,
num_layers=5,
filters=16,
use_batch_norm=True,
load_from_save=False,
    
ultrasound_size = 128
num_classes = 2
min_learning_rate = 0.00001
class_weights = np.array([0.1, 0.9])
learning_rate_decay = (0.002-0.00001) / 100

model = new_unet(
            input_size = ultrasound_size,
            num_classes=num_classes,
            num_channels=num_frames,
            use_batch_norm=use_batch_norm,
            upsample_mode="deconv",  # 'deconv' or 'simple'
            dropout=dropout,
            dropout_type="spatial",
            use_attention=use_attention,
            filters=filters,
            num_layers=num_layers,
            output_activation="softmax",
        )

learning_rate = 0.002
loss_func = weighted_categorical_crossentropy(class_weights)
preprocess_func = train_preprocess
print(learning_rate, learning_rate_decay)
model.compile(
            optimizer=tf.keras.optimizers.Adam(
                lr=learning_rate, decay=learning_rate_decay
            ),
            loss=loss_func,
            metrics=["accuracy", evaluation_metrics.jaccard_coef, evaluation_metrics.dice_coef],
        )

print(model.summary())

(128, 128, 1)
16
0.002 1.99e-05
Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_217 (Conv2D)             (None, 128, 128, 16) 144         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_126 (BatchN (None, 128, 128, 16) 64          conv2d_217[0][0]                 
__________________________________________________________________________________________________
spatial_dropout2d_35 (SpatialDr (None, 128, 128, 16) 0           batch_normalization_126[0][0]    
______________________________________________________